In [1]:
import sys
sys.path.append('../../build')
import cv2 as cv
import numpy as np
import libry as ry
import time
import math
# from perception import find_ball
print(cv.__version__)

4.2.0


In [2]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
#RealWorld.addFile("../../scenarios/challenge.g")
#RealWorld.addFile("../../scenarios/gripper.g")
#RealWorld.addFile("../../scenarios/liftRing.g")
#RealWorld.addFile("../../scenarios/panda_moveGripper.g")
#RealWorld.addFile("../../scenarios/pandasTable.g")
#RealWorld.addFile("../../scenarios/rndObjects.g")
RealWorld.addFile("../../scenarios/panda_paddle.g")

V = ry.ConfigurationViewer()
V.setConfiguration(RealWorld)

In [3]:
# you can also change the shape & size
radius = 0.04

targetObj=RealWorld.addFrame("obj0")
targetObj.setColor([1.,1.,0.])
#targetObj.setShape(ry.ST.sphere, [radius])
targetObj.setShape(ry.ST.capsule, [0.05, 0.05])
#targetObj.setPosition([-0.1, .3, 1.25])
targetObj.setPosition([-0.2, .3, 1.25])
targetObj.setMass(0.1)
targetObj.setContact(1)

In [4]:
V.recopyMeshes(RealWorld)
V.setConfiguration(RealWorld)

# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)

In [5]:
# Frame details print

print("FrameNames : \n", RealWorld.getFrameNames(), "\n");
#print("JointNames : \n",C.getJointNames(), "\n");
#print("getJointDimension : \n",C.getJointDimension(), "\n");
print("GetJointState old : \n", RealWorld.getJointState(), "\n");
print (RealWorld.frame("obj0").getPosition())

#[y,J] = RealWorld.evalFeature(ry.FS.vectorZ, ["paddle"])
#print (y)

#pi = math.pi
#
#q = RealWorld.getJointState()
#q[0] = pi
#q[1] = 0
#q[2] = 1
#q[3] = 0
#
#RealWorld.setJointState(q)
#
#print("GetJointState new : \n", RealWorld.getJointState(), "\n");
#
## updating the display
#V.recopyMeshes(RealWorld)
#V.setConfiguration(RealWorld)

FrameNames : 
 ['panda_link0', 'panda_link0_1', 'panda_link0>panda_joint1', 'panda_joint1', 'panda_link1_1', 'panda_link1>panda_joint2', 'panda_joint2', 'panda_link2_1', 'panda_link2>panda_joint3', 'panda_joint3', 'panda_link3_1', 'panda_link3>panda_joint4', 'panda_joint4', 'panda_link4_1', 'panda_link4>panda_joint5', 'panda_joint5', 'panda_link5_1', 'panda_link5>panda_joint6', 'panda_joint6', 'panda_link6_1', 'panda_link6>panda_joint7', 'panda_joint7', 'panda_link7_1', 'panda_link7>panda_joint8', 'panda_joint8', 'panda_link8>panda_hand_joint', 'frame', 'frame', 'frame', 'frame', 'frame', 'frame', 'frame', 'panda_coll7', 'gripper', 'finger1', 'gripper>finger1', 'finger2', 'gripper>finger2', 'gripperCenter', 'obj0'] 

GetJointState old : 
 [ 0. -1.  0. -2.  0.  2.  0.  0.] 

[-0.2   0.3   1.25]


In [6]:
def grab():
    
    tau = .01
    close = False
    
    print("-------- Grabbing -----------")

    while True:
        if S.getGripperIsGrasping("gripper"): break
        
        time.sleep(tau)
        #grab sensor readings from the simulation
        q = S.get_q()
        
        if not close:
            [y, J] = RealWorld.evalFeature( ry.FS.positionDiff, ["gripperCenter", "obj0"])
            error = np.abs(y).max()
            #print(error)
                                   
            if error < 1.5e-2:
                close = True
                S. closeGripper ("gripper")
                print( "------ Closed Gripper -------")
                
        komo = RealWorld.komo_path(1.,1, tau, True)
        komo. clearObjectives()

        # For smooth q                       
        komo.add_qControlObjective( order=1, scale=1e0)
        # To prevent collisions
        komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e4])
        # Position difference between gripper centre is 0
        komo.addObjective([1.], ry.FS.positionDiff, ['gripperCenter', 'obj0'], ry.OT.sos, [1e1])
        # X axis is parallel
        komo.addObjective([], ry.FS.vectorX, ["gripperCenter"], ry.OT.eq, [1e1], target = [1,0.,0])
        # Y axis is parallel
        komo.addObjective([], ry.FS.vectorY, ["gripperCenter"], ry.OT.eq, [1e1], target = [0.,1,0])
        # To keep gripper open, 
        komo.addObjective([], ry.FS.qItself, ['finger1'], ry.OT.eq, scale=[1e1], order=1)

        komo.optimize()
        
        RealWorld.setFrameState(komo.getConfiguration(0))
        # Get joint state
        q = RealWorld.getJointState()

        # updating the display
        V.recopyMeshes(RealWorld)
        V.setConfiguration(RealWorld)
        
        S.step(q, tau, ry.ControlMode.position)

In [7]:
def wait(timesteps):
    tau = .01

    for t in range(timesteps):
        time.sleep(tau)
        
        S.step([], tau, ry.ControlMode.none)

In [8]:
def liftandthrow():
    throwpoint = 1
    tau = 0.01
    flag = 0
    
    print("------- Lift and Throw ------")
    
    while throwpoint >= 0.5e-2:
        
        [pos, Jpos] = RealWorld.evalFeature( ry.FS.position, ["gripperCenter"])
        throwpoint = np.linalg.norm(pos-[0.25,0.25,1])
        #print(throwpoint)
            
        komo = RealWorld.komo_path(1.,1, tau, True)
        komo. clearObjectives()

        # For smooth q                       
        komo.add_qControlObjective( order=1, scale=1e0)
        # To prevent collisions
        komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e4])
        # Position difference between gripper centre is 0
        komo.addObjective([1.], ry.FS.position, ['gripperCenter'], ry.OT.sos, [1e1], target=[0.25,0.25,1])
        # To keep gripper closed, 
        komo.addObjective([], ry.FS.qItself, ['finger1'], ry.OT.eq, scale=[1e1], order=1)

        if throwpoint <= 0.3 and flag == 0:
            flag = 1
            S.openGripper ("gripper")
            print( "------ Open Gripper -------")    
            
        komo.optimize()
        
        RealWorld.setFrameState(komo.getConfiguration(0))
        # Get joint state
        q = RealWorld.getJointState()

        # updating the display
        V.recopyMeshes(RealWorld)
        V.setConfiguration(RealWorld)
        
        S.step(q, tau, ry.ControlMode.position)

In [9]:
# Grab and Threow the ball
wait(50)
grab()
liftandthrow()
wait(150)

-------- Grabbing -----------
------ Closed Gripper -------
------- Lift and Throw ------
------ Open Gripper -------


# Move to desired position
IK = RealWorld.komo_IK(False)

# Get object position
objpos = RealWorld.frame("obj0").getPosition()
objpos[2] = 0.65
    
# Position difference between paddle centre and ball position
IK.addObjective([1], ry.FS.position, ["paddle"], ry.OT.sos, [1e2], target=objpos)
# Z axis is parallel
IK.addObjective([], ry.FS.vectorZ, ["paddle"], ry.OT.eq, [1e2], target = [0.,0.,-1])
# Qitslef
IK.addObjective([], feature=ry.FS.qItself, type=ry.OT.sos, order=1);
  
IK.optimize(True)

RealWorld.setFrameState( IK.getConfiguration(0) )
V.setConfiguration(RealWorld) #to update your model display

In [18]:
points = []
tau = .01

for t in range(300):
    time.sleep(0.01)
    
    #grab sensor readings from the simulation
    q = S.get_q()
    [distarr,Jdist] = RealWorld.evalFeature(ry.FS.positionDiff, ["paddle", "obj0"])
    dist = np.linalg.norm(distarr)
    print (dist)

    # Get object position
    objpos = RealWorld.frame("obj0").getPosition()
    #objpos[2] = 0.65

    # Move to desired position
    #komo = RealWorld.komo_IK(False)
    # we want to optimize a single step (1 phase, 1 step/phase, duration=1, k_order=1)
    komo = RealWorld.komo_path(1.,1, 0.045, True)

    # addObjective(times, featureSymbol, frameNames, objectiveType, scale, target, order)
    # To prevent collisions
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
    # Position difference between paddle centre and ball position
    komo.addObjective([1], ry.FS.position, ["paddle"], ry.OT.sos, [1e2], target=objpos)
    # Z axis is parallel
    komo.addObjective([], ry.FS.vectorZ, ["paddle"], ry.OT.eq, [1e2], target = [0.,0.,-1])
    # Qitslef
    komo.addObjective([], feature=ry.FS.qItself, type=ry.OT.sos, order=1);

    komo.optimize(True)

    RealWorld.setFrameState(komo.getConfiguration(0))
    # Get joint state
    q = RealWorld.getJointState()

    # updating the display
    V.recopyMeshes(RealWorld)
    V.setConfiguration(RealWorld)

    if(dist <= 0):
        break
        
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

    S.step(q, tau, ry.ControlMode.position)
#   S.step([], tau, ry.ControlMode.none)


0.6084098812419217
0.6153319620454615
0.5099004618950993
0.42762622587494387
0.36379382994199094
0.3140794859042803
0.27481964210952475
0.24311719177483687
0.21683285479497133
0.19438048166998725
0.17467863683390183
0.15701292018346835
0.1409698813582193
0.13157315431587097
0.12621974897194116
0.12236561058417843
0.1192238914939142
0.11647696397155763
0.11397510582037278
0.11163630739565368
0.10944148288538116
0.10738074838557136
0.10544487104672051
0.10362575064559812
0.10191431873006503
0.10030442929287084
0.09878889249489521
0.09735834618076404
0.09600353027178986
0.09473026190732363
0.09352087132914964
0.09238225287583203
0.09129788773884688
0.09027523720468708
0.08929766472324374
0.08837351421807021
0.08748873340359915
0.08664883659575306
0.08584170242392862
0.085074657738778
0.08433422936696923
0.08362887109420232
0.08294584623004674
0.08229306249698219
0.08165881136972052
0.0810498167648546
0.0804562585749865
0.07988794842880984
0.07933111388130563
0.07879497084486513
0.07826927